# `Dataset` sub - class

In [1]:
import os
from glob import glob
from PIL import Image

import torch
from torchvision import datasets, transforms

#### 직접 torch.utils.data.Dataset 상속받아서 데이터셋 구현하기

In [2]:
cifar_dir = '../datasets/cifar/'

In [3]:
os.listdir(cifar_dir)

['labels.txt', 'test', 'test_dataset.csv', 'train', 'train_dataset.csv']

In [4]:
train_dir = cifar_dir + 'train'
test_dir = cifar_dir + 'test'

os.listdir(train_dir)[:3], os.listdir(test_dir)[:3]

(['0_frog.png', '10000_automobile.png', '10001_frog.png'],
 ['0_cat.png', '1000_dog.png', '1001_airplane.png'])

text파일 안에 라벨링이 되어있어 불러오는 작업을 해야한다.

In [8]:
os.path.join(cifar_dir, 'labels.txt')

'../datasets/cifar/labels.txt'

In [10]:
label_list = open(os.path.join(cifar_dir, 'labels.txt'), 'r')
label_list.read()

'airplane\nautomobile\nbird\ncat\ndeer\ndog\nfrog\nhorse\nship\ntruck\n'

In [12]:
with open(os.path.join(cifar_dir, 'labels.txt'), 'r') as f:
    label_list = f.read().strip().split('\n')
label_list

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [13]:
label_list.index('deer')

4

- 불러온 이름라벨들의 인덱스 번호로 숫자라벨을 정해줄 것이다.

In [24]:
train_paths = glob(train_dir + '/*png') # ~~ png로 끝나는 모든 것들
test_paths = glob(test_dir + '/*png') # ~~ png로 끝나는 모든 것들

##### Dataset 클래스 구현

In [33]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_paths, transform=None):
        super(Dataset).__init__()
        self.data_paths = data_paths
        self.transform = transform
    
    def __len__(self, ):
        return len(self.data_paths)
    
    def __getitem__(self, idx):
        path = self.data_paths[idx]    # 하나의 이미지에 대한 경로
        image = Image.open(path)       # 이미지 오픈
        label_name = path.split('.png')[0].split('_')[-1].strip() # 이미지 이름 : 0_frog.png
        label = label_list.index(label_name)
        
        if self.transform: # 별도의 변환 작업이 있다면
            image = self.transform(image)
        
        return image, label
        

##### 디바이스 설정

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

##### 데이터 로더 설정 (custom data)

In [35]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

In [37]:
x, y = next(iter(train_loader))
x.shape, y.shape

(torch.Size([32, 3, 32, 32]), torch.Size([32]))